우리는 앞의 실습에서 Cohesion score, Branching entropy, Accessor Variety 등의 개념적인 코드들을 작성했습니다. 단어 추출 방법의 정답은 없습니다. 반드시 실습 때 썼던 코드와 같이 단어를 추출해야 하는 것도 아닙니다. 여러분이 생각하시기에 적합하다고 생각되는 단어 추출 방법이 있다면 뭐든 쓰셔도 좋습니다. 

정확히는, 우리가 함께 연습했던 Branching entropy는 저는 그대로 쓰고 있지는 않습니다. 앞의 예제에서 Branching entropy 학습 결과는 아래와 같았습니다. 

    단어 (left-side entropy, right-side entropy)
    
    박근	(0.000, 0.082)
    박근혜	(1.510, 2.305)
    박근령	(0.000, 1.335)
    국방	(0.410, 1.097)
    국방부	(-0.000, 1.601)
    국방부는	(0.000, 0.000)
    국방장	(0.000, -0.000)
    국방장관	(0.000, 1.575)
    트와이	(0.000, -0.000)
    트와이스	(0.000, 1.313)
    
트와이스는 실제 단어이기 때문에 left-side entropy 역시 높아야 할 것 같지만, left-side entropy는 0이라고 계산됩니다. 그 이유는 우리가 학습했던 코퍼스가 뉴스데이터였고, 뉴스데이터는 띄어쓰기를 제대로 지키기 때문에 트와이스라는 단어 앞에 다른 단어가 나타난 적이 없기 때문입니다. 

    트와이스는, 트와이스의, ...
    
우리가 코드를 짤 때에는 어절 간의 간격을 고려하지 않았습니다. 

    이번 트와이스의 ...
    음악중심에서 트와이스의 ...
    시청자들은 트와이스에 대하여 ...
    
위와 같이 트와이스라는 단어 앞의 어절에 여러 어절들이 등장할 수 있습니다만, 우리는 하나의 어절에서 L, R만 고려하였기 때문에 트와이스의 left-side entropy가 0이었던 것입니다. 그렇다면 아래와 같이 앞의 어절의 마지막 글자까지 고려하여 entropy를 계산할 수도 있겠습니다. 

    번 [트와이스]의 ...
    서 [트와이스]의 ...
    은 [트와이스]에 ...

아주 간단하게는 문장에서 모든 띄어쓰기를 없엔 다음 모든 substring에 대하여 이를 계산할 수도 있습니다만, 그 경우에는 [번트와이스]와 같은 불필요한 (정확히는 단어가 아닌) subwords까지 고려하게 됩니다. 최대한 띄어쓰기 정보를 보존하며 단어 추출을 하는 것이 더 좋습니다. (좀 더 자세한 이야기는 github.com/lovit/kr-wordrank/tutorials/ 혹은 https://github.com/lovit/KR-WordRank/blob/master/reference/2014_JKIIE_KimETAL_KR-WordRank.pdf 를 참고하십시요)

또한 코드 역시 최적화한 코드가 아닙니다. 그럼에도 불구하고 앞의 예제처럼 실습을 한 이유는 **해당 코드들이 각 알고리즘의 핵심개념**을 익힐 수 있는 좋은 기회이고, 생각보다 **손쉽게 NLP를 할 수 있다는 경험**을 쌓을 수 있기 때문입니다. 용도에 따라서는 앞의 코드를 그대로 써도 좋습니다. 

좀 더 추가적인 후처리나 최적화 등을 더한 코드를 미리 만들어두었습니다. 아래의 주소에 코드를 올려두었으며, pip으로 설치하여 쓰실수도 있습니다. github에 추가적인 작업을 한 뒤, 테스트까지 모두 마치면 version up을 하여 pypi에 올려두고 있습니다.  

    github.com/lovit/soynlp
    
    pip install soynlp
    
실습을 하는 현재의 soynlp의 버전은 0.0.49입니다.

In [1]:
import sys
from config import dataset_dir
sys.path.append('{}/lovit_textmining_dataset'.format(dataset_dir))

soynlp=0.0.49
Dataset version
[navermovie_comments.data] is latest (0.0.1)
[navermovie_comments.models] is latest (0.0.1)
[navernews_10days.data] is latest (0.0.1)
[navernews_10days.models] is latest (0.0.1)


## Corpus

In [3]:
from soynlp.utils import DoublespaceLineCorpus
from navernews_10days import get_news_paths

corpus_path = get_news_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

## WordExtractor (Cohesion score, Branching Entropy, Accessor Variety)

WordExtractor는 Cohesion score, Branching Entropy, Accessor Variety 등을 한번에 계산할 수 있도록 만들어둔 클래스입니다.

In [4]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    max_left_length=10,
    max_right_length=6,
    min_frequency=5
)
word_extractor.train(corpus)

training was done. used memory 0.739 Gbse memory 0.787 Gb


WordExtractor.word_scores()를 하면, 모든 L, R의 subwords에 대하여 앞서 함께 연습한 Cohesion score, Branching Entropy, Accessor Variety, frequency 등을 모두 계산하여 출력하도록 해두었습니다. 여기서 계산하는 Branching Entropy는 어절 간의 글자들도 고려한 수치입니다.

return type은 {word:namedtuple} 형식입니다. Python의 namedtuple 형식이기 때문에 .을 이용하여 해당 값을 손쉽게 가져올 수 있습니다. 

leftside_frequency는 해당 단어가 L에 등장한 횟수이며, rightside_frequency는 해당 단어가 R에 등장한 횟수입니다. 

In [4]:
scores = word_extractor.word_scores()

all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 360721
all accessor variety was computed # words = 360721


In [5]:
scores['뉴스']

Scores(cohesion_forward=0.487322733132789, cohesion_backward=0.22771099423991986, left_branching_entropy=2.877143706774324, right_branching_entropy=3.128831672462708, left_accessor_variety=144, right_accessor_variety=215, leftside_frequency=11340, rightside_frequency=7274)

In [6]:
scores['뉴스'].cohesion_forward

0.487322733132789

In [7]:
for word in ['트와이', '트와이스']:
    lbe = scores[word].left_branching_entropy
    rbe = scores[word].right_branching_entropy
    print(word, '(%.3f, %.3f)' % (lbe, rbe))

트와이 (2.994, -0.000)
트와이스 (2.994, 2.046)


In [8]:
# {word:(cohesion_l, cohesion_r)}
cohesion_scores = word_extractor.all_cohesion_scores()
cohesion_scores['뉴스']

all cohesion probabilities was computed. # words = 223348


(0.487322733132789, 0.22771099423991986)

## Noun extraction

In [9]:
from soynlp.noun import LRNounExtractor
from soynlp.noun import LRNounExtractor_v2
from soynlp.noun import NewsNounExtractor

# noun_extractor = LRNounExtractor()
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(corpus)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 403896 from 223357 sents. mem=0.823 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4434442, mem=1.457 Gb
[Noun Extractor] batch prediction was completed for 146445 words
[Noun Extractor] checked compounds. discovered 69793 compounds
[Noun Extractor] postprocessing detaching_features : 118538 -> 101025
[Noun Extractor] postprocessing ignore_features : 101025 -> 100918
[Noun Extractor] postprocessing ignore_NJ : 100918 -> 100496
[Noun Extractor] 100496 nouns (69793 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.649 Gb                    
[Noun Extractor] 80.22 % eojeols are covered


In [10]:
for word in ['정부', '정부의', '알아', '알아냈', '트와이스', '아이디', '아이디어']:
    print('{}: {}'.format(word, nouns.get(word, '')))

정부: NounScore(frequency=4008, score=0.9993337774816788)
정부의: 
알아: 
알아냈: NounScore(frequency=2, score=1.0)
트와이스: NounScore(frequency=655, score=1.0)
아이디: NounScore(frequency=100, score=1.0)
아이디어: NounScore(frequency=251, score=1.0)


## Tokenization

In [11]:
from soynlp.tokenizer import LTokenizer,MaxScoreTokenizer, RegexTokenizer

cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
ltokenizer = LTokenizer(scores=cohesion_scores)

ltokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다')

['박근혜', '게이트', '에', '대한', '조사', '가', '시작', '되었습니다']

In [12]:
ltokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다', flatten=False)

[('박근혜', ''), ('게이트', '에'), ('대한', ''), ('조사', '가'), ('시작', '되었습니다')]

In [13]:
ltokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다', remove_r=True)

['박근혜', '게이트', '대한', '조사', '시작']

In [14]:
maxscoretokenizer = MaxScoreTokenizer(scores=cohesion_scores)
maxscoretokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다'.replace(' ',''))

['박근혜', '게이트', '에', '대한', '조사', '가', '시작', '되었습니다']

In [15]:
maxscoretokenizer.tokenize('박근혜 게이트에 대한 조사가 시작되었습니다'.replace(' ',''), flatten=False)

[[('박근혜', 0, 3, 0.33444481802269577, 3),
  ('게이트', 3, 6, 0.30753526965628375, 3),
  ('에', 6, 7, 0.0, 1),
  ('대한', 7, 9, 0.1611131928631136, 2),
  ('조사', 9, 11, 0.164128519137783, 2),
  ('가', 11, 12, 0.0, 1),
  ('시작', 12, 14, 0.1257767904844923, 2),
  ('되었습니다', 14, 19, 0.2762976357271788, 5)]]

In [16]:
regextokenizer = RegexTokenizer()
regextokenizer.tokenize('이것123abc유후')

['이것', '123', 'abc', '유후']

In [17]:
from soynlp.tokenizer import normalize

normalize('아닠ㅋㅋㅋㅋㅋㅋㅋ큐ㅠㅠㅠㅠㅠㅠ이럴수가흐규흐규흐규흐규', num_repeat=2)

'아니ㅋㅋㅠㅠ이럴수가흐규흐규'